![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FApplied+GenAI%2FGenerate&file=Long+Context+Retrieval+With+The+Vertex+AI+Gemini+API.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FApplied%2520GenAI%2FGenerate%2FLong%2520Context%2520Retrieval%2520With%2520The%2520Vertex%2520AI%2520Gemini%2520API.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Applied%20GenAI/Generate/Long%20Context%20Retrieval%20With%20The%20Vertex%20AI%20Gemini%20API.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Long Context Retrieval With The Vertex AI Gemini API

**Retrieval - the task of retrieving information as context for an LLM**, like the Gemini family on Vertex AI. Retrieval augmented generation (RAG) is the task of retrieving relevant context and then providing it along with the prompt to the LLM. 

[Long context](https://cloud.google.com/vertex-ai/generative-ai/docs/long-context) is a way of providing full-length sources to the LLM, which can then perform its own retrieval.  Gemini 1.5 Flash (1M) and Gemini 1.5 Pro (2M) have incredible input context windows (1M and 2M tokens respectively) and have shown [near-perfect retrieval of > 99%](https://storage.googleapis.com/deepmind-media/gemini/gemini_v1_5_report.pdf).

For a complete overview of the Gemini API, check out the companion workflow [Vertex AI Gemini API](./Vertex%20AI%20Gemini%20API.ipynb).

**Use Case Exploration**

Buying a home usually involves borrowing money from a lending institution, typically through a mortgage secured by the home's value. But how do these institutions manage the risks associated with such large loans, and how are lending standards established?

In the United States, two government-sponsored enterprises (GSEs) play a vital role in the housing market:
- Federal National Mortgage Association ([Fannie Mae](https://www.fanniemae.com/))
- Federal Home Loan Mortgage Corporation ([Freddie Mac](https://www.freddiemac.com/))

These GSEs purchase mortgages from lenders, enabling those lenders to offer more loans. This process also allows Fannie Mae and Freddie Mac to set standards for mortgages, ensuring they are responsible and borrowers are more likely to repay them. This system makes homeownership more affordable and stabilizes the housing market by maintaining a steady flow of liquidity for lenders and keeping interest rates controlled.

However, navigating the complexities of these GSEs and their extensive servicing guides can be challenging. What if you could directly query these guides and get precise answers without needing to design a complex RAG architecture?

This workflow leverages the long context capabilities of Vertex AI Gemini models and the efficiency of context caching to provide low-latency and cost-effective access to these comprehensive documents. Explore the implementation below!

**References**
- [Retrieval Augmented Generation or Long-Context LLMs? A Comprehensive Study and Hybrid Approach](https://arxiv.org/pdf/2407.16833)


---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [2]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [3]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [4]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform', '1.69.0'),
    ('google.cloud.storage', 'google-cloud-storage')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### API Enablement

In [5]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [6]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)
    IPython.display.display(IPython.display.Markdown("""<div class=\"alert alert-block alert-warning\">
        <b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. The previous cells do not need to be run again⚠️</b>
        </div>"""))

---
## Setup

inputs:

In [7]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [8]:
REGION = 'us-central1'
SERIES = 'applied-genai'
EXPERIMENT = 'long-context'

BUCKET = PROJECT_ID # change to Bucket name if not the same as the Project ID

packages:

In [9]:
# Python standard library imports:
import os, io, base64, json

# package imports
from IPython.display import Markdown
import IPython.display

# vertex ai imports
from google.cloud import aiplatform
import vertexai
import vertexai.generative_models # for Gemini Models

# preview imports for vertex ai api:
from vertexai.preview import caching
import vertexai.preview.generative_models
import vertexai.preview.batch_prediction

# google cloud imports
from google.cloud import storage
import google.auth

In [10]:
aiplatform.__version__

'1.69.0'

clients:

In [11]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)

---
## Gemini Models

Select one of the [supported Gemini models](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#supported-models) and read more about the characteristics of each [here](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).


### Setup Model

Here the [Gemini 1.5 Flash model with version 002](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-1.5-flash) is selected. It has these characteristics (to name a few):
- Max Input Tokens: 1,048,576
- Max Output Tokens: 8,192
- Max image:
    - raw size 20MB
    - base64 encoded size 7MB
    - number per prompt 3000
- Max video:
    - length 1 hour
    - number per prompt 10
- Max audio:
    - length 8.4 hours
    - number per prompt 1
- Max PDF:
    - size 30 MB
- 102 [Languages](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#languages-gemini) for understanding and responding


In [12]:
gemini = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")

### **Prompt With Text**

In [14]:
response = gemini.generate_content('How do I get a mortgage?')
Markdown(response.text)

Getting a mortgage involves a multi-step process. Here's a breakdown of the key steps:

**1. Check Your Credit Score and Report:**

* **Request your credit report:** Get your free annual credit report from each of the three major credit bureaus (Equifax, Experian, and TransUnion) at AnnualCreditReport.com.  Review them for errors and dispute any inaccuracies.
* **Check your credit score:**  Your credit score plays a significant role in mortgage approval and interest rates.  Many credit card companies and banks provide free FICO scores.  Aim for a score of 620 or higher, although higher scores will qualify you for better rates.

**2. Determine How Much You Can Afford:**

* **Use online calculators:** Many mortgage calculators are available online to help you estimate how much you can borrow based on your income, debts, and down payment.
* **Consider your debt-to-income ratio (DTI):**  Lenders look at your DTI, which compares your monthly debt payments to your gross monthly income.  A DTI of 43% or less is generally preferred.
* **Factor in additional costs:** Remember to include property taxes, homeowner's insurance, private mortgage insurance (PMI if your down payment is less than 20%), and potential HOA fees in your budget.

**3. Get Pre-Approved for a Mortgage:**

* **Shop around with multiple lenders:** Compare interest rates, fees, and loan terms from different banks, credit unions, and online mortgage lenders.  Getting pre-approved gives you an idea of how much you can borrow and strengthens your offer when you find a house.
* **Provide necessary documentation:** Lenders will require documentation such as pay stubs, tax returns, bank statements, and W-2s to verify your income and financial stability.

**4. Find a Real Estate Agent (Optional but Recommended):**

* A real estate agent can help you navigate the home-buying process, find properties that meet your needs, and negotiate offers.

**5. Shop for a Home:**

* **Consider your needs and wants:** Think about location, size, amenities, and your overall lifestyle when searching for a home.
* **Attend open houses and schedule private showings:**  Get a feel for different properties and neighborhoods.

**6. Make an Offer on a Home:**

* **Work with your real estate agent to draft a competitive offer:** This includes the purchase price, contingencies (e.g., home inspection, appraisal), and closing date.

**7. Secure Your Financing:**

* **Finalize your mortgage application:** Once your offer is accepted, you'll complete the full mortgage application with your chosen lender.
* **Get a home appraisal:** The lender will order an appraisal to determine the fair market value of the property.
* **Get a home inspection:** A home inspection identifies any potential problems with the property.

**8. Close on Your Mortgage:**

* **Review and sign closing documents:**  This includes the mortgage note, deed, and other legal documents.
* **Pay closing costs:** These include fees for appraisal, title insurance, loan origination, and other services.
* **Receive the keys to your new home!**

**Important Considerations:**

* **Down payment:** While 20% is the traditional down payment, many loan programs offer lower down payment options.  However, if you put down less than 20%, you'll likely have to pay PMI.
* **Loan type:**  Different loan types exist, including fixed-rate mortgages, adjustable-rate mortgages (ARMs), FHA loans, VA loans, and USDA loans.  Each has its own eligibility requirements, terms, and benefits.
* **Interest rates:** Interest rates fluctuate and can significantly impact your monthly payments.


This process can seem daunting, but breaking it down into these manageable steps can make it less overwhelming.  Don't hesitate to ask questions throughout the process and seek professional advice from lenders, real estate agents, and financial advisors.


Fannie:
https://selling-guide.fanniemae.com/
Freddie:
https://guide.freddiemac.com/ci/okcsFattach/get/1002095_2